In [18]:
import datetime as dt
import random
import logging

import numpy as np
import pandas as pd
import kquant as kq

In [ ]:
# API_account
def set_api_account():
    kq.set_api("KRX2308020", "EQDkUcyI3dK6oIAXqAR8BXOK4bKxHHmH")
    return None

set_api_account()

In [20]:
class STATUS_LOADER:
    def __init__(self, dict_df_result, dict_df_position) -> None:
        self.dict_df_result = dict_df_result
        self.dict_df_position = dict_df_position

    def get_current_cash(self):
        _dict_df_result = self.dict_df_result
        try:
            _df_result_total = _dict_df_result["TOTAL"]
            _current_cash = (
                _df_result_total.sort_values("DATE").tail(1)["CASH"].values[0]
            )
            return _current_cash
        except:
            return 1_000_000_000

    def get_status_df(self):
        current_symbol_list = list()
        _dict_df_result = self.dict_df_result
        _dict_df_position = self.dict_df_position

        _total_symbols = sorted(_dict_df_position.keys())

        for _symbol in _total_symbols:
            try:
                _symbol_result_df = _dict_df_result[_symbol]
                _symbol_position_df = _dict_df_position[_symbol]

                _current_price = (
                    _symbol_result_df.sort_values("DATE").tail(1)["PRICE"].values[0]
                )
                _trade_price = _symbol_position_df["TRADE_PRICE"].values[0]
                _current_qty = _symbol_position_df["QTY"].values[0]

                current_symbol_list.append(
                    {
                        "SYMBOL": _symbol,
                        "CURRENT_QTY": _current_qty,
                        "CURRENT_PRICE": _current_price,
                        "TRADE_PRICE": _trade_price,
                    }
                )
            except:
                pass
        return pd.DataFrame(
            current_symbol_list,
            columns=["SYMBOL", "CURRENT_QTY", "CURRENT_PRICE", "TRADE_PRICE"],
        )

In [157]:
dict_df_result = dict()
dict_df_position = dict()

status_loader = STATUS_LOADER(dict_df_result, dict_df_position)

current_cash = status_loader.get_current_cash()
daily_invest_money = current_cash / 2
status_df = status_loader.get_status_df()

position_symbols = sorted(set(status_df["SYMBOL"]))

In [21]:
class SYMBOL_LOADER:
    @staticmethod
    def load_symbols_df():
        symbols_df = kq.symbol_stock()
        return symbols_df

    class SYMBOL_FILTER:
        @staticmethod
        def filter__market(symbols_df):
            filtered_symbols_df = symbols_df[
                (symbols_df["MARKET"].isin(["코스닥", "유가증권"]))
            ].copy()
            return filtered_symbols_df

        @staticmethod
        def filter__admin_issue(symbols_df):
            filtered_symbols_df = symbols_df[(symbols_df["ADMIN_ISSUE"] == 0)].copy()
            return filtered_symbols_df

        @staticmethod
        def filter_sec_type(symbols_df):
            filtered_symbols_df = symbols_df[
                symbols_df["SEC_TYPE"].isin(["ST", "EF", "EN"])
            ].copy()
            return filtered_symbols_df

    def filter_symbols_df(self, symbols_df):
        symbol_filter = self.SYMBOL_FILTER()
        filtered_symbols_df = symbol_filter.filter__market(symbols_df)
        filtered_symbols_df = symbol_filter.filter__admin_issue(filtered_symbols_df)
        filtered_symbols_df = symbol_filter.filter_sec_type(filtered_symbols_df)
        return filtered_symbols_df

    @staticmethod
    def get_symbols(symbols_df):
        symbols = sorted(set(symbols_df["SYMBOL"]))
        return symbols

    # SYMBOL_LOADER PIPELINE
    def __call__(self):
        symbols_df = self.load_symbols_df()
        filtered_symbols_df = self.filter_symbols_df(symbols_df)
        symbols = self.get_symbols(filtered_symbols_df)
        return symbols

In [13]:
symbol_loader = SYMBOL_LOADER()
total_symbols = symbol_loader()

sampled_symbols = random.sample(total_symbols, 20)
using_symbols = sorted(set(sampled_symbols + position_symbols))

In [22]:
class FUNDAMENTAL_LOADER:
    def __init__(self, symbol, date) -> None:
        self.symbol = symbol
        self.date = date
        self.daily_stock_df = kq.daily_stock(
            symbol,
            start_date=date - dt.timedelta(days=7),
            end_date=date,
        )

    def load_recent_close(self):
        daily_stock_df = self.daily_stock_df
        _close = daily_stock_df.sort_values("DATE").tail(1)["CLOSE"].values[0]
        return _close

    def load_recent_marketcap(self):
        daily_stock_df = self.daily_stock_df
        _marketcap = daily_stock_df.sort_values("DATE").tail(1)["MARKETCAP"].values[0]
        return _marketcap

    def load_recent_netprofit(self):
        netprofit_df = kq.account_history(self.symbol, "122700")
        netprofit_df.sort_values("YEARMONTH", inplace=True)
        _netprofit = netprofit_df.tail(1)["VALUE"].values[0]
        return _netprofit

    def load_recent_capital(self):
        capital_df = kq.account_history(self.symbol, "115000")
        capital_df.sort_values("YEARMONTH", inplace=True)
        _capital = capital_df.tail(1)["VALUE"].values[0]
        return _capital

    def __call__(self):
        _close = self.load_recent_close()
        _marketcap = self.load_recent_marketcap()
        _netprofit = self.load_recent_netprofit()
        _capital = self.load_recent_capital()
        return {
            "SYMBOL": self.symbol,
            "CLOSE": _close,
            "MARKETCAP": _marketcap,
            "NETPROFIT": _netprofit,
            "CAPITAL": _capital,
        }

In [14]:
import datetime as dt

In [15]:
date = dt.date(2023,1,2)

fundamental_data_list = list()
for symbol in using_symbols:
    try:
        _fundamental_loader = FUNDAMENTAL_LOADER(symbol, date)
        _fundamental_data = _fundamental_loader()
        fundamental_data_list.append(_fundamental_data)
    except:
        pass

fundamental_df = pd.DataFrame(fundamental_data_list)

In [16]:
fundamental_df

,SYMBOL,CLOSE,MARKETCAP,NETPROFIT,CAPITAL
0,002460,"10,150","103,022,500,000","22,672,300","356,952,449"
1,004150,"3,185","133,797,317,745","48,952,154","597,332,919"
2,005810,"27,550","286,782,496,400","73,235,786","964,559,570"
3,009180,"2,725","74,875,983,125","24,240,799","102,505,681"
4,011150,"2,905","104,378,895,565","4,066,803","33,421,151"
5,011700,"6,000","194,676,906,000","-4,733,469","92,946,406"
6,015890,"6,260","182,971,975,000","37,464,888","420,836,852"
7,027710,"1,595","177,709,477,000","-7,550,762","249,388,172"
8,027740,"1,190","75,578,361,320","655,477","48,976,891"
9,033560,"3,700","63,270,000,000","-264,689","160,357,894"


In [23]:
""" 
symbols_and_orders
"""


class FUDAMENTAL_PROCESSOR:
    def __init__(self, fundamental_df) -> None:
        fundamental_df["PBR"] = fundamental_df["MARKETCAP"] / (
            fundamental_df["CAPITAL"] * 1000
        )
        fundamental_df = fundamental_df[fundamental_df["PBR"] > 0]
        self.fundamental_df = fundamental_df

    class GET_BUYING_ORDERS:
        def __init__(
            self, fundamental_processor, daily_invest_money, position_symbols
        ) -> None:
            self.fundamental_processor = fundamental_processor
            self.daily_invest_money = daily_invest_money
            self.position_symbols = position_symbols

        @staticmethod
        def filter_position_symbols(fundamental_df, position_symbols):
            filtered_fundamental_df = fundamental_df[
                ~(fundamental_df["SYMBOL"].isin(position_symbols))
            ]
            return filtered_fundamental_df

        @staticmethod
        def get_low_pbr_df(fundamental_df):
            low_pbr_df = fundamental_df.nsmallest(5, "PBR")
            return low_pbr_df

        @staticmethod
        def append_pbr_weight(low_pbr_df):
            low_pbr_df["PBR_WEIGHT"] = low_pbr_df["PBR"].sum() / low_pbr_df["PBR"]
            return low_pbr_df

        @staticmethod
        def append_price_invest(low_pbr_df, daily_invest_money):
            low_pbr_df["PRICE_INVEST"] = (
                low_pbr_df["PBR_WEIGHT"] / low_pbr_df["PBR_WEIGHT"].sum()
            ) * daily_invest_money
            return low_pbr_df

        @staticmethod
        def append_cnt_invest(low_pbr_df):
            low_pbr_df["CNT_INVEST"] = low_pbr_df["PRICE_INVEST"] // low_pbr_df["CLOSE"]
            return low_pbr_df

        def __call__(self):
            fundamental_df = self.fundamental_processor.fundamental_df
            daily_invest_money = self.daily_invest_money
            position_symbols = self.position_symbols

            filtered_fundamental_df = self.filter_position_symbols(
                fundamental_df, position_symbols
            )
            low_pbr_df = self.get_low_pbr_df(filtered_fundamental_df)
            low_pbr_df = self.append_pbr_weight(low_pbr_df)
            low_pbr_df = self.append_price_invest(low_pbr_df, daily_invest_money)
            low_pbr_df = self.append_cnt_invest(low_pbr_df)
            buying_orders = list(
                low_pbr_df.set_index("SYMBOL")["CNT_INVEST"]
                .astype(int)
                .to_dict()
                .items()
            )
            return buying_orders

    class GET_SELLING_ORDERS:
        def __init__(self, fundamental_processor, status_df) -> None:
            self.fundamental_processor = fundamental_processor
            self.status_df = status_df

        @staticmethod
        def get_limit_line(fundamental_df):
            # limit_line : PBR 상위 75 %
            limit_line = np.percentile(fundamental_df["PBR"], 75)
            return limit_line

        @staticmethod
        def get_high_pbr_df(fundamental_df, limit_line):
            high_pbr_df = fundamental_df[fundamental_df["PBR"] > limit_line]
            return high_pbr_df

        @staticmethod
        def filter_position_symbols(high_pbr_df, position_symbols):
            filtered_position_symbols = sorted(
                set(high_pbr_df["SYMBOL"]) & set(position_symbols)
            )
            return filtered_position_symbols

        def __call__(self):
            fundamental_df = self.fundamental_processor.fundamental_df
            status_df = self.status_df
            position_symbols = sorted(set(status_df["SYMBOL"]))

            limit_line = self.get_limit_line(fundamental_df)
            high_pbr_df = self.get_high_pbr_df(fundamental_df, limit_line)
            filtered_position_symbols = self.filter_position_symbols(
                high_pbr_df, position_symbols
            )

            selling_df = status_df[status_df["SYMBOL"].isin(filtered_position_symbols)]

            selling_orders = list(
                selling_df.set_index("SYMBOL")["CURRENT_QTY"]
                .apply(lambda x: x * -1)
                .astype(int)
                .to_dict()
                .items()
            )
            return selling_orders

In [17]:
fundamental_processor = FUDAMENTAL_PROCESSOR(fundamental_df)

In [166]:
get_buying_orders = FUDAMENTAL_PROCESSOR.GET_BUYING_ORDERS(
    fundamental_processor, daily_invest_money
)

In [167]:
buying_orders = get_buying_orders()

In [168]:
get_selling_orders = FUDAMENTAL_PROCESSOR.GET_SELLING_ORDERS(
    fundamental_processor, status_df
)

In [169]:
selling_orders = get_selling_orders()

[('375500', 4923),
 ('163560', 19684),
 ('009160', 17706),
 ('016600', 189711),
 ('009300', 4216)]

In [24]:
def trade_func(
    date: dt.date,
    dict_df_result: dict[str, pd.DataFrame],
    dict_df_position: dict[str, pd.DataFrame],
    logger: logging.Logger,
) -> list[tuple[str, int]]:
    """
    STATUS_LOADER
        : get_current_cash()
            -> 현재 가용 가능한 현금을 가져옵니다.
        : get_status_df()
            -> 현재 포지션이 있는 주식들에 대한 정보를 가져옵니다.
    """
    status_loader = STATUS_LOADER(dict_df_result, dict_df_position)

    current_cash = status_loader.get_current_cash()
    daily_invest_money = current_cash / 2
    status_df = status_loader.get_status_df()
    position_symbols = sorted(set(status_df["SYMBOL"]))

    """
    SYMBOL_LOADER
        : __call__()
            -> 현재 시장에서 거래 가능한 symbol을 모두 가져옵니다.
    """
    symbol_loader = SYMBOL_LOADER()
    total_symbols = symbol_loader()

    sampled_symbols = random.sample(total_symbols, 100)
    using_symbols = sorted(set(sampled_symbols + position_symbols))
    """
    FUNDAMENTAL_LOADER
        : __call__()
            -> 특정 symbol에 대하여, fundamental anlysis를 위해 필요한 데이터를 추출합니다.
    """

    fundamental_data_list = list()
    for symbol in using_symbols:
        try:
            _fundamental_loader = FUNDAMENTAL_LOADER(symbol, date)
            _fundamental_data = _fundamental_loader()
            fundamental_data_list.append(_fundamental_data)
        except:
            pass
    fundamental_df = pd.DataFrame(fundamental_data_list)

    """
    FUDAMENTAL_PROCESSOR
        : GET_BUYING_ORDERS
        : GET_SELLING_ORDERS
    """
    fundamental_processor = FUDAMENTAL_PROCESSOR(fundamental_df)

    get_buying_orders = FUDAMENTAL_PROCESSOR.GET_BUYING_ORDERS(
        fundamental_processor, daily_invest_money, position_symbols
    )
    buying_orders = get_buying_orders()
    get_selling_orders = FUDAMENTAL_PROCESSOR.GET_SELLING_ORDERS(
        fundamental_processor, status_df
    )
    selling_orders = get_selling_orders()

    symbols_and_orders = buying_orders + selling_orders
    return symbols_and_orders

In [25]:
# 초기 매매일 코드
dict_df_result, dict_df_position, logger = kq.backtest_stock_port_daily(
    trade_func,
    "2023-01-02",
    "2023-01-02",
    init_cash=1_000_000_000,
    return_position=True,
    return_logger=True,
)

[2023-01-02] 종목: 003300, 주문전 보유수량:      0 주문수량: 16,682, 매매수량: 16,682, 주문후 보유수량: 16,682
[2023-01-02] 종목: 126560, 주문전 보유수량:      0 주문수량: 37,173, 매매수량: 37,173, 주문후 보유수량: 37,173
[2023-01-02] 종목: 109960, 주문전 보유수량:      0 주문수량: 38,557, 매매수량: 38,557, 주문후 보유수량: 38,557
[2023-01-02] 종목: 079960, 주문전 보유수량:      0 주문수량:  4,987, 매매수량:  4,987, 주문후 보유수량:  4,987
[2023-01-02] 종목: 234100, 주문전 보유수량:      0 주문수량: 35,945, 매매수량: 35,945, 주문후 보유수량: 35,945


In [31]:
dict_df_result = kq.backtest_stock_port_daily(
    trade_func,
    "2023-01-02",  # 실제 심사에서는 투자기간 시작일
    "2023-01-31",  # 실제 심사에서는 투자기간 종료일
    init_cash=1_000_000_000,  # 10억원
)

[2023-01-02] 종목: 001390, 주문전 보유수량:      0 주문수량: 51,521, 매매수량: 51,521, 주문후 보유수량: 51,521
[2023-01-02] 종목: 071320, 주문전 보유수량:      0 주문수량:  3,304, 매매수량:  3,304, 주문후 보유수량:  3,304
[2023-01-02] 종목: 000320, 주문전 보유수량:      0 주문수량:  8,991, 매매수량:  8,991, 주문후 보유수량:  8,991
[2023-01-02] 종목: 015750, 주문전 보유수량:      0 주문수량: 14,281, 매매수량: 14,281, 주문후 보유수량: 14,281
[2023-01-02] 종목: 002350, 주문전 보유수량:      0 주문수량:  7,732, 매매수량:  7,732, 주문후 보유수량:  7,732
[2023-01-03] 종목: 001390, 주문전 보유수량: 51,521 주문수량: 23,976, 매매수량: 23,976, 주문후 보유수량: 75,497
[2023-01-03] 종목: 071320, 주문전 보유수량:  3,304 주문수량:  1,597, 매매수량:  1,597, 주문후 보유수량:  4,901
[2023-01-03] 종목: 000320, 주문전 보유수량:  8,991 주문수량:  4,429, 매매수량:  4,429, 주문후 보유수량: 13,420
[2023-01-03] 종목: 032190, 주문전 보유수량:      0 주문수량:  1,044, 매매수량:  1,044, 주문후 보유수량:  1,044
[2023-01-03] 종목: 015750, 주문전 보유수량: 14,281 주문수량:  6,624, 매매수량:  6,624, 주문후 보유수량: 20,905
[2023-01-04] 종목: 001390, 주문전 보유수량: 75,497 주문수량: 11,682, 매매수량: 11,682, 주문후 보유수량: 87,179
[2023-01-04] 종목: 071320, 주문전 보유수량:  4,901 주

In [32]:
dict_df_result['TOTAL']

,DATE,SYMBOL,PRICE,ORDER,QTY,TRADE_PRICE,POSITION,AVG_PRICE,FEE,TRADE_TAX,SLIPPAGE,CASHFLOW,CASH,HIST_VALUE,STOCK_VALUE,TOTAL_VALUE,REAL_PROFIT,UNREAL_PROFIT,PROFIT,HIGHWATERMARK,DRAWDOWN
0,2023-01-02,TOTAL,0,0,0,0,"85,829",0.0000,0,0,0,"-499,979,940","500,020,060","499,979,940","499,979,940","1,000,000,000",0,0,0,"1,000,000,000",0
1,2023-01-03,TOTAL,0,0,0,0,"123,499",0.0000,0,0,0,"-249,969,710","250,050,350","749,949,650","750,892,805","1,000,943,155",0,"943,155","943,155","1,000,943,155",0
2,2023-01-04,TOTAL,0,0,0,0,"139,738",0.0000,0,0,0,"-124,986,360","125,063,990","874,936,010","886,256,320","1,011,320,310",0,"11,320,310","11,320,310","1,011,320,310",0
3,2023-01-05,TOTAL,0,0,0,0,"146,890",0.0000,0,0,0,"-61,830,120","63,233,870","936,766,130","965,873,460","1,029,107,330",0,"29,107,330","29,107,330","1,029,107,330",0
4,2023-01-06,TOTAL,0,0,0,0,"150,304",0.0000,0,0,0,"-31,462,500","31,771,370","968,228,630","1,010,784,585","1,042,555,955",0,"42,555,955","42,555,955","1,042,555,955",0
5,2023-01-09,TOTAL,0,0,0,0,"153,487",0.0000,0,0,0,"-15,494,460","16,276,910","983,723,090","1,052,250,330","1,068,527,240",0,"68,527,240","68,527,240","1,068,527,240",0
6,2023-01-10,TOTAL,0,0,0,0,"154,978",0.0000,0,0,0,"-7,513,080","8,763,830","991,236,170","1,061,954,870","1,070,718,700",0,"70,718,700","70,718,700","1,070,718,700",0
7,2023-01-11,TOTAL,0,0,0,0,"155,773",0.0000,0,0,0,"-4,305,990","4,457,840","995,542,160","1,067,808,785","1,072,266,625",0,"72,266,625","72,266,625","1,072,266,625",0
8,2023-01-12,TOTAL,0,0,0,0,"155,998",0.0000,0,0,0,"-2,151,370","2,306,470","997,693,530","1,069,360,150","1,071,666,620",0,"71,666,620","71,666,620","1,072,266,625","600,005"
9,2023-01-13,TOTAL,0,0,0,0,"156,121",0.0000,0,0,0,"-1,030,930","1,275,540","998,724,460","1,076,632,425","1,077,907,965",0,"77,907,965","77,907,965","1,077,907,965",0
